<a href="https://colab.research.google.com/github/Nithyarajoman/Machine-learning-Trial/blob/main/Library_Functions/MAC_Data_Calculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

In [2]:
# MNIST Fashion dataset for the example cnn model
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float')/255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float')/255

4422102/4422102 [==============================] - 0s 0us/step


In [17]:
# ImageNet classification dataset for Resnet18 and ResNet50
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar100.load_data() #Size is (, 32,32,3)

169001437/169001437 [==============================] - 4s 0us/step


In [3]:
# Create Model
model_hierarchy = models.Sequential(
    [        
        layers.Conv2D(32, kernel_size=(3, 3),
                      input_shape=(28, 28, 1),
                      strides=(2,2), padding="same",
                      use_bias=False),
        layers.BatchNormalization(),
        layers.Activation(activation="relu"),
        #layers.MaxPooling2D(pool_size=(2, 2)),
        models.Sequential([
            layers.Conv2D(64, kernel_size=(3, 3),
                          strides=(2,2), padding="valid",
                          use_bias=False),
            layers.BatchNormalization(),
            layers.Activation(activation="relu")
            ]),            
        #layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ]
)

In [4]:
# Train
model_hierarchy.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model_hierarchy.fit(x=x_train, y=y_train, epochs=5, batch_size=128, validation_split=0.1 )
score = model_hierarchy.evaluate(x_test, y_test, verbose=0)

Epoch 1/5
422/422 [==============================] - 20s 44ms/step - loss: 0.5621 - accuracy: 0.8000 - val_loss: 0.6253 - val_accuracy: 0.8018
Epoch 2/5
422/422 [==============================] - 18s 43ms/step - loss: 0.3839 - accuracy: 0.8617 - val_loss: 0.3328 - val_accuracy: 0.8770
Epoch 3/5
422/422 [==============================] - 18s 44ms/step - loss: 0.3431 - accuracy: 0.8775 - val_loss: 0.2876 - val_accuracy: 0.8967
Epoch 4/5
422/422 [==============================] - 19s 45ms/step - loss: 0.3208 - accuracy: 0.8844 - val_loss: 0.2839 - val_accuracy: 0.8942
Epoch 5/5
422/422 [==============================] - 18s 43ms/step - loss: 0.3012 - accuracy: 0.8909 - val_loss: 0.2747 - val_accuracy: 0.8993


In [5]:
# Function to calculate return MAC from each layer

def mac_each_layer(l):
    o_shape, i_shape, strides, ks, filters = ['', '', ''], ['', '', ''], [1, 1], [0, 0], [0, 0]        
    macc = 0
    
    if ('Dense' in str(l)):
        i_shape = l.input.shape[1:4].as_list()[0]
        o_shape = l.output.shape[1:4].as_list()
        macc = (o_shape[0] * i_shape)

    
    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' not in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((filters * ks[0] * ks[1] * i_shape[2]) * (
                (i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    if ('Conv2D ' in str(l) and 'DepthwiseConv2D' in str(l) and 'SeparableConv2D' not in str(l)):
        strides = l.strides
        ks = l.kernel_size
        filters = l.filters
        i_shape = l.input.get_shape()[1:4].as_list()
        o_shape = l.output.get_shape()[1:4].as_list()

        if (filters == None):
            filters = i_shape[2]

        macc = ((ks[0] * ks[1] * i_shape[2]) * ((i_shape[0] / strides[0]) * (i_shape[1] / strides[1])))

    return macc

In [6]:
# Function to pass layers to the calculating function and to get the macc of each layer

def MAC_summary(model):
  print('%25s | %6s' % ('Layer Name', 'MACs'))
  print('*' * 100)
  t_macc = 0

  for l in model.layers:
    if (type(l) in [models.Sequential, keras.Model]):
      for ib_layer in l.layers:
        mac_obtained = mac_each_layer(ib_layer)
        print('%25s(%5s) | %5.4f' % (
                l.name,ib_layer.name, mac_obtained ))
        t_macc += mac_obtained  

    else:
      mac_obtained = mac_each_layer(l)
      print('%25s | %5.4f' % (
                l.name, mac_obtained ))
      
    t_macc += mac_obtained
  print('\nTotal MACs operations in the complete model: %10.8f\n' % (t_macc))
  return

In [13]:
# Function to print the effective data communicated 

def data_summary(model):

  # Function to add communicated data to the metrics
  def add_metrics(m2):
    "Annotates recursiverly a (non-compiled) model with metrics that calculate probability of non-zero activity"
    prefix_name = m2.name + "_"
    for ll in m2.layers:
        if is_hier_layer(ll):
            add_metrics(ll)
        else:
          if not ("BatchNormalization" in str(ll)):
            my_metric = tf.reduce_sum( tf.math.count_nonzero(ll.output, dtype=tf.int32) / tf.size( ll.output)  )
            name=prefix_name + ll.name
            m2.add_metric(my_metric, name=name)
    return

  # Function to check if a layer is model or not
  def is_hier_layer(layer):
      "Finds if layer is actually a model instead of a single layer"
      return type(layer) in [models.Sequential, keras.Model]

  # Clone the model. Note that it is not compiled!
  m2 = keras.models.clone_model(model)
  add_metrics(m2)
  
  # Compiling the cloned model and taking the inference
  m2.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  aux = m2.evaluate(x_test[0:1000], y_test[0:1000]) 
  
  print('%40s | %20s' % ('Metrics and Layer Name', 'Effective data communicated'))
  print("*" * 150)
  for l in m2.metrics:
    print('%40s | %5.4f' % (l.name, l.result().numpy()))
  return

In [11]:
# MAC calculation for sample cnn
MAC_summary(model_hierarchy)

               Layer Name |   MACs
****************************************************************************************************
                   conv2d | 56448.0000
      batch_normalization | 0.0000
               activation | 0.0000
               sequential(conv2d_1) | 903168.0000
               sequential(batch_normalization_1) | 0.0000
               sequential(activation_1) | 0.0000
                  flatten | 0.0000
                  dropout | 0.0000
                    dense | 23040.0000

Total MACs operations in the complete model: 982656.00000000



In [14]:
# Effective calculation for sample cnn
data_summary(model_hierarchy)

32/32 [==============================] - 1s 8ms/step - loss: 2.3117 - accuracy: 0.0660 - sequential_conv2d_1: 0.8909 - sequential_activation_1: 0.3996 - sequential_1_conv2d: 0.6556 - sequential_1_activation: 0.3371 - sequential_1_flatten: 0.3996 - sequential_1_dropout: 0.3996 - sequential_1_dense: 1.0000
                  Metrics and Layer Name | Effective data communicated
******************************************************************************************************************************************************
                                    loss | 2.3117
                                accuracy | 0.0660
                     sequential_conv2d_1 | 0.8909
                 sequential_activation_1 | 0.3996
                     sequential_1_conv2d | 0.6556
                 sequential_1_activation | 0.3371
                    sequential_1_flatten | 0.3996
                    sequential_1_dropout | 0.3996
                      sequential_1_dense | 1.0000


In [16]:
# Calculations for Resnet18 CNN model
!pip install image-classifiers
from tensorflow.keras.applications import imagenet_utils
from classification_models.keras import Classifiers

ResNet18, preprocess_input = Classifiers.get('resnet18')
model_ResNet18 = ResNet18(include_top=False,
                        input_tensor=None,
                        input_shape=(
                            32, 32, 3),
                        pooling=None)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 50 kB 4.0 MB/s 


In [18]:
MAC_summary(model_ResNet18)

               Layer Name |   MACs
****************************************************************************************************
                     data | 0.0000
                  bn_data | 0.0000
           zero_padding2d | 0.0000
                    conv0 | 3396288.0000
                      bn0 | 0.0000
                    relu0 | 0.0000
         zero_padding2d_1 | 0.0000
                 pooling0 | 0.0000
         stage1_unit1_bn1 | 0.0000
       stage1_unit1_relu1 | 0.0000
         zero_padding2d_2 | 0.0000
       stage1_unit1_conv1 | 3686400.0000
         stage1_unit1_bn2 | 0.0000
       stage1_unit1_relu2 | 0.0000
         zero_padding2d_3 | 0.0000
       stage1_unit1_conv2 | 3686400.0000
          stage1_unit1_sc | 262144.0000
                      add | 0.0000
         stage1_unit2_bn1 | 0.0000
       stage1_unit2_relu1 | 0.0000
         zero_padding2d_4 | 0.0000
       stage1_unit2_conv1 | 3686400.0000
         stage1_unit2_bn2 | 0.0000
       stage1_unit2_relu2 | 0.

In [19]:
data_summary(model_ResNet18)

32/32 [==============================] - 8s 93ms/step - loss: 12.3672 - accuracy: 0.0120 - model_data: 0.9966 - model_zero_padding2d: 0.7067 - model_conv0: 0.9997 - model_relu0: 0.5333 - model_zero_padding2d_1: 0.4213 - model_pooling0: 0.6755 - model_stage1_unit1_relu1: 0.6755 - model_zero_padding2d_2: 0.4323 - model_stage1_unit1_conv1: 1.0000 - model_stage1_unit1_relu2: 0.4492 - model_zero_padding2d_3: 0.2875 - model_stage1_unit1_conv2: 1.0000 - model_stage1_unit1_sc: 0.9999 - model_add: 1.0000 - model_stage1_unit2_relu1: 0.4563 - model_zero_padding2d_4: 0.2921 - model_stage1_unit2_conv1: 1.0000 - model_stage1_unit2_relu2: 0.5749 - model_zero_padding2d_5: 0.3679 - model_stage1_unit2_conv2: 1.0000 - model_add_1: 1.0000 - model_stage2_unit1_relu1: 0.4479 - model_zero_padding2d_6: 0.2867 - model_stage2_unit1_conv1: 1.0000 - model_stage2_unit1_relu2: 0.5099 - model_zero_padding2d_7: 0.2266 - model_stage2_unit1_conv2: 1.0000 - model_stage2_unit1_sc: 1.0000 - model_add_2: 1.0000 - model_sta

In [20]:
# Calculations for Resnet50 CNN model
from tensorflow.keras.applications.resnet50 import ResNet50
model_Resnet50 = ResNet50(include_top=False,
                        weights='imagenet',
                        input_shape=(32, 32, 3),
                        pooling=None)

94765736/94765736 [==============================] - 1s 0us/step


In [21]:
MAC_summary(model_Resnet50)

               Layer Name |   MACs
****************************************************************************************************
                  input_1 | 0.0000
                conv1_pad | 0.0000
               conv1_conv | 3396288.0000
                 conv1_bn | 0.0000
               conv1_relu | 0.0000
                pool1_pad | 0.0000
               pool1_pool | 0.0000
      conv2_block1_1_conv | 262144.0000
        conv2_block1_1_bn | 0.0000
      conv2_block1_1_relu | 0.0000
      conv2_block1_2_conv | 2359296.0000
        conv2_block1_2_bn | 0.0000
      conv2_block1_2_relu | 0.0000
      conv2_block1_0_conv | 1048576.0000
      conv2_block1_3_conv | 1048576.0000
        conv2_block1_0_bn | 0.0000
        conv2_block1_3_bn | 0.0000
         conv2_block1_add | 0.0000
         conv2_block1_out | 0.0000
      conv2_block2_1_conv | 1048576.0000
        conv2_block2_1_bn | 0.0000
      conv2_block2_1_relu | 0.0000
      conv2_block2_2_conv | 2359296.0000
        conv2_bloc

In [22]:
data_summary(model_Resnet50)

32/32 [==============================] - 17s 187ms/step - loss: 11.1866 - accuracy: 0.0000e+00 - resnet50_input_1: 0.9966 - resnet50_conv1_pad: 0.7067 - resnet50_conv1_conv: 0.9997 - resnet50_conv1_relu: 0.5427 - resnet50_pool1_pad: 0.4288 - resnet50_pool1_pool: 0.7098 - resnet50_conv2_block1_1_conv: 0.9999 - resnet50_conv2_block1_1_relu: 0.5159 - resnet50_conv2_block1_2_conv: 1.0000 - resnet50_conv2_block1_2_relu: 0.5183 - resnet50_conv2_block1_0_conv: 0.9999 - resnet50_conv2_block1_3_conv: 1.0000 - resnet50_conv2_block1_add: 1.0000 - resnet50_conv2_block1_out: 0.5406 - resnet50_conv2_block2_1_conv: 1.0000 - resnet50_conv2_block2_1_relu: 0.5138 - resnet50_conv2_block2_2_conv: 1.0000 - resnet50_conv2_block2_2_relu: 0.4727 - resnet50_conv2_block2_3_conv: 1.0000 - resnet50_conv2_block2_add: 1.0000 - resnet50_conv2_block2_out: 0.7257 - resnet50_conv2_block3_1_conv: 1.0000 - resnet50_conv2_block3_1_relu: 0.5908 - resnet50_conv2_block3_2_conv: 1.0000 - resnet50_conv2_block3_2_relu: 0.6057 -